# Limpeza dos dados no dataset Titanic

## Importando as bibliotecas necessárias

In [38]:
from matplotlib import pyplot as plt
from sklearn.utils import resample
import seaborn as sns
import pandas as pd
import numpy as np

## Link para o dataset principal

[Dataset principal - main.ipynb](./main.ipynb)

## Importando os Datasets

In [39]:
df_titanic_raw = pd.read_csv("./databases/raw/titanic.csv", sep = ",", index_col = "PassengerId")
df_coracao_raw = pd.read_csv("./databases/raw/heart-disease.csv", sep = ",")

## Breve apresentacao do dataset

In [40]:
df_titanic_raw.sample(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
67,1,2,"Nye, Mrs. (Elizabeth Ramell)",female,29.0,0,0,C.A. 29395,10.5000,F33,S
463,0,1,"Gee, Mr. Arthur H",male,47.0,0,0,111320,38.5000,E63,S
814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
319,1,1,"Wick, Miss. Mary Natalie",female,31.0,0,2,36928,164.8667,C7,S
336,0,3,"Denkoff, Mr. Mitto",male,NaN,0,0,349225,7.8958,NaN,S
146,0,2,"Nicholls, Mr. Joseph Charles",male,19.0,1,1,C.A. 33112,36.7500,NaN,S
523,0,3,"Lahoud, Mr. Sarkis",male,NaN,0,0,2624,7.2250,NaN,C
141,0,3,"Boulos, Mrs. Joseph (Sultana)",female,NaN,0,2,2678,15.2458,NaN,C


In [41]:
df_titanic_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


No dataset Titanic, podemos ver que há dados faltantes em diversas colunas.

Será realizado um análise e um possível limpeza nesses dados.

In [42]:
df_coracao_raw.sample(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
177,64,1,2,140,335,0,1,158,0,0.0,2,0,2,0
284,61,1,0,140,207,0,0,138,1,1.9,2,1,3,0
45,52,1,1,120,325,0,1,172,0,0.2,2,0,2,1
17,66,0,3,150,226,0,1,114,0,2.6,0,0,2,1
34,51,1,3,125,213,0,0,125,1,1.4,2,1,2,1
43,53,0,0,130,264,0,0,143,0,0.4,1,0,2,1
69,62,0,0,124,209,0,1,163,0,0.0,2,0,2,1
81,45,1,1,128,308,0,0,170,0,0.0,2,0,2,1
229,64,1,2,125,309,0,1,131,1,1.8,1,0,3,0
33,54,1,2,125,273,0,0,152,0,0.5,0,1,2,1


In [43]:
df_coracao_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [44]:
print("Valores máximos:")
for coluna in df_coracao_raw.columns:
    print(coluna, end = "\t")
    print(df_coracao_raw[coluna].max())
    
print("\n\nValores mínimos:")
for coluna in df_coracao_raw.columns:
    print(coluna, end = "\t")
    print(df_coracao_raw[coluna].min())


Valores máximos:
age	77
sex	1
cp	3
trestbps	200
chol	564
fbs	1
restecg	2
thalach	202
exang	1
oldpeak	6.2
slope	2
ca	4
thal	3
target	1


Valores mínimos:
age	29
sex	0
cp	0
trestbps	94
chol	126
fbs	0
restecg	0
thalach	71
exang	0
oldpeak	0.0
slope	0
ca	0
thal	0
target	0


No dataset Doenças do Coração, podemos ver que não há dados faltantesem nenhuma das colunas. Todavia os tipos dos dados estão incorretos. 

Será realizado uma troca nos tipos dos dados deste dataset

## Limpeza dos dados no dataset Titanic

In [45]:
df_titanic_raw.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

### Verificando colunas problemáticas

In [46]:
print(df_titanic_raw["Age"].head(10))
print(df_titanic_raw["Cabin"].head(10))
print(df_titanic_raw["Embarked"].head(10))

PassengerId
1     22.0
2     38.0
3     26.0
4     35.0
5     35.0
6      NaN
7     54.0
8      2.0
9     27.0
10    14.0
Name: Age, dtype: float64
PassengerId
1      NaN
2      C85
3      NaN
4     C123
5      NaN
6      NaN
7      E46
8      NaN
9      NaN
10     NaN
Name: Cabin, dtype: object
PassengerId
1     S
2     C
3     S
4     S
5     S
6     Q
7     S
8     S
9     S
10    C
Name: Embarked, dtype: object


Foi após análise visual, foi decidido:
- deletar a coluna "Cabin";
- preencher a coluna "Embarked";
    - Utilizando a moda.
- preenchar a coluna "Age".
    - Utilizando a mediana.

### Realizando mudanças no dataset

In [47]:
df_titanic_raw["Embarked"].mode()

0    S
Name: Embarked, dtype: object

In [48]:
df_titanic_processed = df_titanic_raw.drop(columns = ["Cabin"], axis = "columns")
df_titanic_processed["Age"] = df_titanic_processed["Age"].fillna(df_titanic_processed["Age"].median())
df_titanic_processed["Embarked"] = df_titanic_processed["Embarked"].fillna("S")
df_titanic_processed.isna().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

### Descartando colunas vistas como inúteis para a análise dos dados

In [49]:
colunas_para_descartar = [
    "Parch",
    "Ticket"
]

In [50]:
df_titanic_processed.drop(columns = colunas_para_descartar, axis = "column", inplace = True)

### Mudanças nos tipos do dado do dataset Titanic

In [51]:
df_titanic_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


As colunas:
- Survived;
- Pclass;
- Sex;
- Embarked.

Serão classificadas como tipo "categórico"(category).

As colunas:
- SibSp;
- Age.

Serão classificadas como tipo "inteiro"(uint8).

A coluna "Fare" será classificada como tipo ponto flutuante (float32).

A coluna "Name" será classificada como tipo alfanumérico (str).

In [52]:
df_titanic_processed["Survived"]    = df_titanic_processed["Survived"].astype("category")
df_titanic_processed["Pclass"]      = df_titanic_processed["Pclass"].astype("category")
df_titanic_processed["Name"]        = df_titanic_processed["Name"].astype("string")
df_titanic_processed["Sex"]         = df_titanic_processed["Sex"].astype("category")
df_titanic_processed["Age"]         = df_titanic_processed["Age"].astype("uint8")
df_titanic_processed["SibSp"]       = df_titanic_processed["SibSp"].astype("uint8")
df_titanic_processed["Fare"]        = df_titanic_processed["Fare"].astype("float32")
df_titanic_processed["Embarked"]    = df_titanic_processed["Embarked"].astype("category")

#### Verificando mudança

In [53]:
df_titanic_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    category
 1   Pclass    891 non-null    category
 2   Name      891 non-null    string  
 3   Sex       891 non-null    category
 4   Age       891 non-null    uint8   
 5   SibSp     891 non-null    uint8   
 6   Fare      891 non-null    float32 
 7   Embarked  891 non-null    category
dtypes: category(4), float32(1), string(1), uint8(2)
memory usage: 23.1 KB


### Mudanças no valores da coluna "Sex"

In [54]:
df_titanic_processed["Sex"].unique()

['male', 'female']
Categories (2, object): ['female', 'male']

Para melhor manipulação do dataset, foi escolhido manter os dados categóricos seguindo a
convenção de números inteiros, pequenos e positivos, como é possível notar nas colunas
"Survived" e "Pclass".

Portanto será realizado uma manipulação dos dados, de forma que:
- ` female ` será substituido pelo número ` 0 `;
- ` male ` será substituido pelo número ` 1 `.

In [55]:
novos_valores_Sex = {
    "female": 0,
    "male": 1
}

In [56]:
df_titanic_processed["Sex"] = df_titanic_processed["Sex"].map(novos_valores_Sex)
df_titanic_processed["Sex"].unique()

[1, 0]
Categories (2, int64): [0, 1]

### Mudanças no valores da coluna "Embarked"

In [57]:
df_titanic_processed["Embarked"].unique()

['S', 'C', 'Q']
Categories (3, object): ['C', 'Q', 'S']

Para melhor manipulação do dataset, foi escolhido manter os dados categóricos seguindo a
convenção de números inteiros, pequenos e positivos, como é possível notar nas colunas
"Sex", "Survived" e "Pclass".

Portanto será realizado uma manipulação dos dados, de forma que:
- ` C ` será substituido pelo número ` 0 `;
- ` Q ` será substituido pelo número ` 1 `;
- ` S ` será substituido pelo número ` 2 `.

In [58]:
novos_valores_Embarked = {
    "C": 0,
    "Q": 1,
    "S": 2
}

In [59]:
df_titanic_processed["Embarked"] = df_titanic_processed["Embarked"].map(novos_valores_Embarked)
df_titanic_processed["Embarked"].unique()

[2, 0, 1]
Categories (3, int64): [0, 1, 2]

### Verificando se os dados da coluna "Pclass" estão balanceados

In [60]:
df_titanic_processed["Pclass"].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

Como podemos ver acima, os dados estão muito desbalanceados.

Para uma melhor análise, será preciso reabalancear os dados, ou para mais ou para menos.
Se balancearmos para menos, perderemos uma quantidade enorme de dados, então foi escolhido balancear para mais.

#### Gerando vários datasets para balancear os dados

##### Hiper parametros

In [61]:
tamanho_das_classes = df_titanic_processed[df_titanic_processed["Pclass"] == 3].shape[0]
semente_randomica = 42

##### Realizando a criação

In [62]:
df_Pclass_1 = df_titanic_processed[df_titanic_processed["Pclass"] == 1]
df_Pclass_2 = df_titanic_processed[df_titanic_processed["Pclass"] == 2]
df_Pclass_3 = df_titanic_processed[df_titanic_processed["Pclass"] == 3]

##### Realizando o balanceamento

In [63]:
df_Pclass_1 = resample(df_Pclass_1,
                       replace = True,
                       n_samples = tamanho_das_classes,
                       random_state = semente_randomica)
df_Pclass_2 = resample(df_Pclass_2,
                       replace = True,
                       n_samples = tamanho_das_classes,
                       random_state = semente_randomica)

#### Unindo os datasets em um só

In [64]:
df_titanic_processed_balanceado = pd.concat([df_Pclass_3, df_Pclass_1, df_Pclass_2])
df_titanic_processed_balanceado["Pclass"].value_counts()

Pclass
1    491
2    491
3    491
Name: count, dtype: int64

### Salvando o dataset tratado em um pickle (.pkl)

In [65]:
df_titanic_processed_balanceado.to_pickle("./databases/processed/titanic_processado.pkl")

## Tratamento do tipo dos dados no dataset Doenças do Coração

### Copiando o dataframe original para outro dataframe

In [66]:
df_coracao_processed = df_coracao_raw

### Mudando os tipos dos dados das colunas

In [67]:
df_coracao_processed["age"]         = df_coracao_processed["age"].astype("uint8")
df_coracao_processed["sex"]         = df_coracao_processed["sex"].astype("category")
df_coracao_processed["cp"]          = df_coracao_processed["cp"].astype("category")
df_coracao_processed["trestbps"]    = df_coracao_processed["trestbps"].astype("uint8")
df_coracao_processed["chol"]        = df_coracao_processed["chol"].astype("uint16")
df_coracao_processed["fbs"]         = df_coracao_processed["fbs"].astype("category")
df_coracao_processed["restecg"]     = df_coracao_processed["restecg"].astype("category")
df_coracao_processed["thalach"]     = df_coracao_processed["thalach"].astype("uint8")
df_coracao_processed["exang"]       = df_coracao_processed["exang"].astype("category")
df_coracao_processed["oldpeak"]     = df_coracao_processed["oldpeak"].astype("float32")
df_coracao_processed["slope"]       = df_coracao_processed["slope"].astype("category")
df_coracao_processed["ca"]          = df_coracao_processed["ca"].astype("category")
df_coracao_processed["thal"]        = df_coracao_processed["thal"].astype("category")
df_coracao_processed["target"]      = df_coracao_processed["target"].astype("category")

#### Verificando mudança

In [68]:
df_coracao_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       303 non-null    uint8   
 1   sex       303 non-null    category
 2   cp        303 non-null    category
 3   trestbps  303 non-null    uint8   
 4   chol      303 non-null    uint16  
 5   fbs       303 non-null    category
 6   restecg   303 non-null    category
 7   thalach   303 non-null    uint8   
 8   exang     303 non-null    category
 9   oldpeak   303 non-null    float32 
 10  slope     303 non-null    category
 11  ca        303 non-null    category
 12  thal      303 non-null    category
 13  target    303 non-null    category
dtypes: category(9), float32(1), uint16(1), uint8(3)
memory usage: 6.8 KB


Agora vamos balancear os dados conforme a coluna "restecg".

Esse novo tratamento será realizado pois no notebook de análise dos dados, foi possível notar que esta coluna está muito desbalanceada.

### Gerando vários datasets para balancear os dados

#### Hiper parametros

In [69]:
tamanho_das_classes = df_coracao_processed[df_coracao_processed["restecg"] == 1].shape[0]
semente_randomica = 42

#### Realizando a criação

In [70]:
df_restecg_0 = df_coracao_processed[df_coracao_processed["restecg"] == 0]
df_restecg_1 = df_coracao_processed[df_coracao_processed["restecg"] == 1]
df_restecg_2 = df_coracao_processed[df_coracao_processed["restecg"] == 2]

#### Realizando o balanceamento

In [71]:
df_restecg_0 = resample(df_restecg_0,
                       replace = True,
                       n_samples = tamanho_das_classes,
                       random_state = semente_randomica)
df_restecg_2 = resample(df_restecg_2,
                       replace = True,
                       n_samples = tamanho_das_classes,
                       random_state = semente_randomica)

#### Unindo os datasets em um só

In [72]:
df_coracao_processed_balanceado = pd.concat([df_restecg_1, df_restecg_0, df_restecg_2])
df_coracao_processed_balanceado["restecg"].value_counts()

restecg
0    152
1    152
2    152
Name: count, dtype: int64

### Salvando o dataset tratado em um pickle (.pkl)

In [73]:
df_coracao_processed_balanceado.to_pickle("./databases/processed/coracao_processado.pkl")